In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation,Reshape,Convolution2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
os.chdir("/Dataset")
numbers=os.listdir()

if os.path.isdir("train/0") is False:
    os.mkdir("train")
    os.mkdir("test")
    os.mkdir("valid")
    
    for i in numbers:
        shutil.move(f"{i}","train")
        os.mkdir(f"valid/{i}")
        os.mkdir(f"test/{i}")
        for k in random.sample(os.listdir(f"train/{i}"),30):
            shutil.move(f"train/{i}/{k}",f"valid/{i}")
        for j in random.sample(os.listdir(f"train/{i}"),5):
            shutil.move(f"train/{i}/{j}",f"test/{i}")
os.chdir("..")


In [ ]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
train_batches=train_batches.flow_from_directory(
    directory="train",batch_size=10,target_size=(224,224))
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory="valid",batch_size=10,target_size=(224,224))
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory="test",batch_size=10,target_size=(224,224),shuffle=False)

In [4]:
assert train_batches.n==1712
assert valid_batches.n==300
assert test_batches.n==50
assert test_batches.num_classes==valid_batches.num_classes==train_batches.num_classes


In [ ]:
mobile=tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

In [6]:
x=mobile.layers[-5].output
output=tf.keras.layers.Flatten()(x)
outputs=Dense(units=10,activation="softmax")(output)


In [7]:
model=Model(inputs=mobile.input,outputs=outputs)

In [ ]:
for layer in model.layers[0:-23]:
    layer.trainable=False
model.summary()


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(x=train_batches,validation_data=valid_batches,epochs=30,verbose=2)
model.save("/model.keras")

In [ ]:

model1=load_model("model.keras")
model1.summary()



In [12]:
predictions=model1.predict(test_batches,verbose=0)
rounded=np.round(predictions)
cm=confusion_matrix(y_true=test_batches.classes,y_pred=np.argmax(rounded,axis=-1))


In [ ]:
display=ConfusionMatrixDisplay(cm)
display.plot()
accuracy_score(test_batches.classes,np.argmax(rounded,axis=-1))